In [ ]:
!pip install pandas numpy scipy matplotlib seaborn scikit-learn keras tensorflow

In [ ]:
!pip install tensorflow==2.18

# Исследование предсказуемости потребительского поведения по оценке Колмогоровской сложности

In [3]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from scipy.linalg import hankel

import tensorflow as tf 
from tensorflow import keras
from keras.layers import LSTM, Dropout, Dense

from sklearn.model_selection import train_test_split

2024-12-11 17:55:59.774154: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-12-11 17:56:00.043105: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1733928960.165775 3066931 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1733928960.197614 3066931 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-11 17:56:00.503013: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [2]:
!pip install tensorflow[and-cuda]==2.18.0

In [4]:
print(len(tf.config.list_physical_devices('GPU')))

1


In [5]:
from lempel_ziv import *
from huffman import *
from utils import *

In [6]:
data = pd.read_csv('data/transact_18_22.csv', parse_dates=[2])
data.head(3)

,client,card,date,amt,mcc,group,value
0,390989,3048567,2018-01-01,878.0,5814,food,survival
1,1309946,3226559,2018-01-01,249.0,5411,food,survival
2,1386278,2715990,2018-01-01,50.0,5499,food,survival


## Предварительная обработка данных

In [18]:
filtered_data = data[(data['date'] >= '2019-01-15') & (data['date'] <= '2019-12-14')]
filtered_data.head(3)

,client,card,date,amt,mcc,group,value
3278420,1378858,2391344,2019-01-15,350.87,5499,food,survival
3278421,3103680,3187433,2019-01-15,2660.00,5331,nonfood,socialization
3278422,2136426,3762147,2019-01-15,485.00,5331,nonfood,socialization


In [19]:
filtered_data['value'].unique()

array(['survival', 'socialization', 'money', 'self_realization', '7372',
       '4411', '2741', '5681', '6211', '8249', '5094', '5983', '7278',
       '7699', '5734', '7210', '7998', '7393', '5310', '3008', '5074',
       '7216', '7251', '5950', '5713', '7629', '5047', '5733', '7929',
       '7394', '3301', '5111', '3211', '5946', '8351', '5697', '5611',
       '5131', '7622', '5976', '5970', '7379', '5655', '6513', '4214',
       '5967', '5122', '5521', '5932', '5065', '5044', '5013', '7375',
       '5811', '5933', '5099', '8111', '5072', '8661', '5300', '5532',
       '7395', '8641', '5818', '5994', '8050', '7535', '5971', '7338',
       '5045', '7996', '5172', '5051', '5817', '7333', '4582', '3010',
       '5021', '5137', '7911', '7392', '3586', '7933', '5231', '5718',
       '780', '7297', '7032', '5085', '1711', '8211', '1799', '7534',
       '8699', '5046', '5940', '1731', '7829', '3553', '4119', '3042',
       '5960', '3005', '5169', '5972', '5571', '3533', '3831', '7273',
     

In [20]:
values_to_keep = ['socialization', 'survival', 'money', 'self_realization']
filtered_data = filtered_data[filtered_data['value'].isin(values_to_keep)]

In [21]:
filtered_data.head(3)

,client,card,date,amt,mcc,group,value
3278420,1378858,2391344,2019-01-15,350.87,5499,food,survival
3278421,3103680,3187433,2019-01-15,2660.00,5331,nonfood,socialization
3278422,2136426,3762147,2019-01-15,485.00,5331,nonfood,socialization


In [ ]:
grouped = filtered_data.groupby(['client', 'date'])
new_data = []
for (client, date), group in grouped:
    row = {'client': client, 'date': date}
    for val in values_to_keep:
        row[val] = 1 if val in group['value'].values else 0  
    new_data.append(row)

new_df = pd.DataFrame(new_data)
new_df['code'] = new_df[['socialization', 'survival', 'money', 'self_realization']].apply(binary_to_hex, axis=1)

In [ ]:
new_df.head(5)

In [7]:
#new_df.to_csv('output.csv')
new_df = pd.read_csv('output.csv', parse_dates=[2], index_col=0)
new_df.head(3)

,client,date,socialization,survival,money,self_realization,code
0,224,2019-01-15,1,0,0,0,8
1,224,2019-01-16,1,1,0,1,d
2,224,2019-01-17,1,1,0,0,c


## Алгоритм оценки Колмогоровской сложности 

### Пример работы для одного клиента

In [8]:
def calculate_f1_accuracy(x_pred, x_real):
    """Вычисляет F1-меру и точность.

    Args:
        y_pred: Предсказанные бинарные метки (массив NumPy).
        y_true: Истинные бинарные метки (массив NumPy).

    Returns:
        Кортеж: (F1-мера, точность). Возвращает (0, 0), если нет положительных предсказаний.
    """
    x_pred, x_real = x_pred.astype(int), x_real.astype(int) 
    tp = len(np.where(x_pred[np.where(x_real == 1)] == 1)[0])
    tn = len(np.where(x_pred[np.where(x_real == 0)] == 0)[0])
    fp = len(np.where(x_pred[np.where(x_real == 0)] == 1)[0])
    fn = len(np.where(x_pred[np.where(x_real == 1)] == 0)[0])
    if (tp + fp) * (tp + fn) * tp:
        precision, recall = tp / (tp + fp), tp / (tp + fn)
        f1 = 2 * precision * recall / (precision + recall) 
    elif sum(x_pred - x_real):
        f1 = 0.
    else:
        f1 = 1.
    if (tp + tn + fp + fn):
        accuracy = (tp + tn) / (tp + tn + fp + fn) * 100
    else:
        accuracy = 0.
    return f1, accuracy

In [32]:
selected_columns = ['date', 'socialization', 'survival', 'money', 'self_realization', 'code']
client_224_transactions = new_df[new_df['client'] == 1421807].loc[:, selected_columns]

In [33]:
client_224_transactions.shape

(328, 6)

In [34]:
text = client_224_transactions['code'].values
huffman_codes = huffman_code(text)
encoded_string = ''.join(huffman_codes[symbol] for symbol in text)

In [35]:
print("Результаты кодирования Хаффмана:")
print("-" * 30)
print(f"Закодированная строка:\n{encoded_string}")
print("Таблица кодов Хаффмана:")
print(pd.DataFrame.from_dict(huffman_codes, orient='index', columns=['Код Хаффмана']))
complexity = len(encoded_string) / len(text) / 4.0
print(f"Сложность: {complexity:.3f}")

Результаты кодирования Хаффмана:
------------------------------
Закодированная строка:
1111101100010010010111110100010110000000000001110100011111000010100100000000000100000010101101101010001110010111000001010100011111110101001010100000111000110001000101101010101100000001010010111001001001101001010000111011001001110101111111111100101110100110000000010000011010011000001000000000110000011000001110000110000110101110011001001000011001101001011101010100110000101100010010110011010001000010001100010010010000010100110011010001010011100110010001001010101001100000000000100011010101010100100000000100100000100110010011011011010110111010101010001010011011000101010000001001001000110001001000110000010011000100001001001110110100110100101100100101100110100011000100110100010010101001001101010101001001111000010110100000001001100001000101010101011000110101100111000000011010001011100010010100101110000
Таблица кодов Хаффмана:
  Код Хаффмана
4            1
c           01
e         0011
d         0010
f       

In [ ]:
text = client_224_transactions['survival'].values
forecast_horizon = 7 
lzw_depth = 52   
X, y = create_dataset(text, lzw_depth, forecast_horizon)
X_train, y_train = X[:150], y[:150]
X_test, y_test=X[150:], y[150:]
X.shape, y.shape

((286, 28, 8), (286, 7))

In [43]:
model = keras.Sequential([
    LSTM(X_train.shape[1], return_sequences=True),
    Dropout(0.3),
    LSTM(X_train.shape[1], return_sequences=False),
    Dense(X_train.shape[1], activation='tanh'),
    Dense(y_train.shape[1], activation='sigmoid')
])

In [44]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [45]:
model.fit(X_train, y_train, epochs=15, batch_size=1, verbose=0) 

In [40]:
predictions = model.predict(X_test) 

for i in range(y_test.shape[1]):
    f1, accuracy = calculate_f1_accuracy(predictions[:, i].round(), y_test[:, i])
    print(f'День {i+1}: F1={f1:.2f}; Точность: {accuracy:.1f}%')

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 106ms/step
День 1: F1=1.00; Точность: 100.0%
День 2: F1=1.00; Точность: 100.0%
День 3: F1=1.00; Точность: 100.0%
День 4: F1=1.00; Точность: 100.0%
День 5: F1=0.99; Точность: 98.1%
День 6: F1=0.99; Точность: 98.1%
День 7: F1=1.00; Точность: 100.0%


### Пример работы для нескольких клиентов

In [26]:
split = 52  # Индекс, с которого начинаем прогнозирование

In [51]:
'''results_df = pd.DataFrame(
        columns=['client_id'] + [f'f1_{horizon}' for horizon in forecast_horizons] +
    [f'huffman_{col}' for col in predicted_columns]
)
results_df['client_id'] = client_ids  
'''
result_df = pd.read_csv('out_1.csv', index_col=0)
result_df.head(5)

,client_id,f1_1,f1_7,f1_14,f1_28,huffman_complexity
0,2190618,0.910891,0.822222,0.924731,0.886076,0.037086
1,208625,0.952381,0.960000,0.936170,0.964706,0.042282
2,3483687,1.000000,1.000000,NaN,NaN,0.022654
3,3441340,NaN,NaN,NaN,NaN,0.034859
4,1421807,NaN,NaN,NaN,NaN,0.038348


In [52]:
lzw_depth = 56

In [53]:
clients = list(new_df['client'].value_counts().index)

In [54]:
#num_clients = len(clients)
num_clients = len(clients) - 7
forecast_horizons = [1, 7, 14, 28]  
start = 7

In [55]:
from keras.backend import clear_session

In [56]:
res=pd.DataFrame(columns=['id']+['f1'+str(i) for i in forecast_horizons])
res['id']=['id']*num_clients

In [ ]:
with tf.device('/GPU:0'):
    for i in range(start, start + num_clients):
        client_transactions = new_df[new_df['client'] == clients[i]].loc[:, selected_columns]
        text = client_transactions['socialization'].values

        for j, forecast_horizon in enumerate(forecast_horizons):
            X, y = create_dataset(text, 28, forecast_horizon)
            X_train, y_train = X[:split], y[:split]
            X_test, y_test = X[split:split + forecast_horizon], y[split:split + forecast_horizon]

            clear_session()
            model = keras.Sequential([
                LSTM(X_train.shape[1], return_sequences=True),
                Dropout(0.3),
                LSTM(X_train.shape[1], return_sequences=False),
                Dense(X_train.shape[1], activation='tanh'),
                Dense(y_train.shape[1], activation='sigmoid')
            ])

            model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

            model.fit(X_train, y_train, epochs=15, batch_size=1, verbose=0) 
            
            predictions = model.predict(X_test) 
            f1, accuracy = calculate_f1_accuracy(predictions[:,-1].round(0).astype(int),  y_test[:,-1])
            result_df.loc[result_df['client_id'] == clients[i], f'f1_{forecast_horizon}'] = f1
        if i % 5 == 0:
            result_df.to_csv('out_1.csv')
        del model

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 94ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 91ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 94ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 93ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 95ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 97ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 93ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 94ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 95ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 95ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 100ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 96ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 96ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 95ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 93ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 95ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 94ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 94ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 92ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 92ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 92ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 95ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 94ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 98ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 93ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 92ms/step
1/1 ━━━━━━━

In [ ]:
result_df.head(10)

In [ ]:
result_df.to_csv('out_1.csv')

In [ ]:
results_df = pd.DataFrame(
        columns=['client_id'] + [f'f1_{horizon}' for horizon in forecast_horizons] +
    [f'huffman_complexity']
)
results_df['client_id'] = list(data['client'].value_counts().index) 

In [ ]:
with tf.device('/GPU:0'):
    for i in range(num_clients+20):
        client_transactions = new_df[new_df['client'] == clients[i]].loc[:, selected_columns]
        text = client_transactions['survival'].values

        for forecast_horizon in forecast_horizons:
            X, y = create_dataset(text, lzw_depth, forecast_horizon)
            X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

            model = keras.Sequential([
                LSTM(X_train.shape[1], return_sequences=True),
                Dropout(0.3),
                LSTM(X_train.shape[1], return_sequences=False),
                Dense(X_train.shape[1], activation='tanh'),
                Dense(y_train.shape[1], activation='sigmoid')
            ])

            model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

            model.fit(X_train, y_train, epochs=15, batch_size=32, verbose=0) 
            
            predictions = model.predict(X_test) 
            f1, accuracy = calculate_f1_accuracy(predictions[:, -1].round().astype(int), y_test[:, -1])
            result_df.loc[result_df['client_id'] == clients[i], f'f1_{forecast_horizon}'] = f1
            print(f'f1_{forecast_horizon} for client_{clients[i]} = {f1}')
        del model

In [ ]:
result_df.to_csv('out_2.csv')

In [ ]:
results_df = pd.DataFrame(
        columns=['client_id'] + [f'f1_{horizon}' for horizon in forecast_horizons] +
    [f'huffman_complexity']
)
results_df['client_id'] = list(data['client'].value_counts().index) 

In [ ]:
with tf.device('/GPU:0'):
    for i in range(num_clients+20):
        client_transactions = new_df[new_df['client'] == clients[i]].loc[:, selected_columns]
        text = client_transactions['money'].values

        for forecast_horizon in forecast_horizons:
            X, y = create_dataset(text, lzw_depth, forecast_horizon)
            X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

            model = keras.Sequential([
                LSTM(X_train.shape[1], return_sequences=True),
                Dropout(0.3),
                LSTM(X_train.shape[1], return_sequences=False),
                Dense(X_train.shape[1], activation='tanh'),
                Dense(y_train.shape[1], activation='sigmoid')
            ])

            model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

            model.fit(X_train, y_train, epochs=15, batch_size=32, verbose=0) 
            
            predictions = model.predict(X_test) 
            f1, accuracy = calculate_f1_accuracy(predictions[:, -1].round().astype(int), y_test[:, -1])
            result_df.loc[result_df['client_id'] == clients[i], f'f1_{forecast_horizon}'] = f1
            print(f'f1_{forecast_horizon} for client_{clients[i]} = {f1}')
        del model

In [ ]:
result_df.to_csv('out_3.csv')

In [ ]:
results_df = pd.DataFrame(
        columns=['client_id'] + [f'f1_{horizon}' for horizon in forecast_horizons] +
    [f'huffman_complexity']
)
results_df['client_id'] = list(data['client'].value_counts().index) 

In [ ]:
with tf.device('/GPU:0'):
    for i in range(num_clients+20):
        client_transactions = new_df[new_df['client'] == clients[i]].loc[:, selected_columns]
        text = client_transactions['self_realization'].values

        for forecast_horizon in forecast_horizons:
            X, y = create_dataset(text, lzw_depth, forecast_horizon)
            X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

            model = keras.Sequential([
                LSTM(X_train.shape[1], return_sequences=True),
                Dropout(0.3),
                LSTM(X_train.shape[1], return_sequences=False),
                Dense(X_train.shape[1], activation='tanh'),
                Dense(y_train.shape[1], activation='sigmoid')
            ])

            model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

            model.fit(X_train, y_train, epochs=15, batch_size=32, verbose=0) 
            
            predictions = model.predict(X_test) 
            f1, accuracy = calculate_f1_accuracy(predictions[:, -1].round().astype(int), y_test[:, -1])
            result_df.loc[result_df['client_id'] == clients[i], f'f1_{forecast_horizon}'] = f1
            print(f'f1_{forecast_horizon} for client_{clients[i]} = {f1}')
        del model

In [ ]:
result_df.to_csv('out_4.csv')

In [ ]:
results_df = pd.DataFrame(
        columns=['client_id'] + [f'f1_{horizon}' for horizon in forecast_horizons] +
    [f'huffman_complexity']
)
results_df['client_id'] = list(data['client'].value_counts().index) 

In [ ]:
num_clients = len(clients)

In [ ]:
with tf.device('/GPU:0'):
    for i in range(num_clients):
        client_transactions = new_df[new_df['client'] == clients[i]].loc[:, selected_columns]
        text = client_transactions['code'].values

        for forecast_horizon in forecast_horizons:
            X, y = create_dataset(text, lzw_depth, forecast_horizon)
            X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

            model = keras.Sequential([
                LSTM(X_train.shape[1], return_sequences=True),
                Dropout(0.3),
                LSTM(X_train.shape[1], return_sequences=False),
                Dense(X_train.shape[1], activation='tanh'),
                Dense(y_train.shape[1], activation='sigmoid')
            ])

            model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

            model.fit(X_train, y_train, epochs=15, batch_size=32, verbose=0) 
            
            predictions = model.predict(X_test) 
            f1, accuracy = calculate_f1_accuracy(predictions[:, -1].round().astype(int), y_test[:, -1])
            result_df.loc[result_df['client_id'] == clients[i], f'f1_{forecast_horizon}'] = f1
            print(f'f1_{forecast_horizon} for client_{clients[i]} = {f1}')
        del model

In [ ]:
result_df.to_csv('out_5.csv')

не забудь обучить первые 20 клиентов